In [30]:
import pandas as pd
import numpy as np
benign_df = pd.read_csv("benign_domain_list.csv", header=None)
malicious_df = pd.read_csv("Malicious_URLs.csv")
mixed_df = pd.read_csv("mixed_domain_list.csv", header=None)
tranco_df = pd.read_csv("tranco_full_list_for_ranking.csv", header=None)

In [2]:
benign_df

,0
0,google.com
1,facebook.com
2,a-msedge.net
3,youtube.com
4,microsoft.com
...,...
4958,bethesda.net
4959,shopifysvc.com
4960,instana.io
4961,oanda.com


In [3]:
malicious_df

,url
0,https://ervashipping.com/
1,http://www.gabung-grupmabarcimbrut.duckdns.org/
2,http://republik-jancuk.000webhostapp.com/
3,https://www.chet-notnkt8-grup.duckdns.org/
4,http://veromcertiom.ddns.net/connexion/certico...
...,...
10624,http://sites.google.com/site/habbotuttogratis
10625,http://www.habbocreditosparati.blogspot.com/
10626,http://creditiperhabbogratissicuro100.blogspot...
10627,http://mundovirtualhabbo.blogspot.com/2009_01_...


In [4]:
mixed_df

,0
0,goodrx.com
1,weather.com.cn
2,draftkings.com
3,suntimes.com
4,moe.gov.cn
...,...
178,services.runescape.com-ul.cc
179,nsu-amazon.1inc.top
180,80fxv9.cn
181,nsu-amazon.3inc.top


In [5]:
tranco_df

,0,1
0,1,google.com
1,2,facebook.com
2,3,a-msedge.net
3,4,youtube.com
4,5,microsoft.com
...,...,...
3648682,3648683,jeventanddesign.com
3648683,3648684,medi-verbund.de
3648684,3648685,stjudemedical.net
3648685,3648686,dsa-vorschau.de


In [6]:
for row in benign_df[0]:
    benign_df['Domain Length'] = benign_df[0].str.len()
benign_df 


,0,Domain Length
0,google.com,10
1,facebook.com,12
2,a-msedge.net,12
3,youtube.com,11
4,microsoft.com,13
...,...,...
4958,bethesda.net,12
4959,shopifysvc.com,14
4960,instana.io,10
4961,oanda.com,9


In [7]:
from urllib.parse import urlparse

def get_fqdn(url):
    parsed_url = urlparse(url)
    return parsed_url.netloc

malicious_df['Domain'] = malicious_df['url'].apply(get_fqdn)

for row in malicious_df['Domain']:
    malicious_df['Domain Length'] = malicious_df['Domain'].str.len()
malicious_df

,url,Domain,Domain Length
0,https://ervashipping.com/,ervashipping.com,16
1,http://www.gabung-grupmabarcimbrut.duckdns.org/,www.gabung-grupmabarcimbrut.duckdns.org,39
2,http://republik-jancuk.000webhostapp.com/,republik-jancuk.000webhostapp.com,33
3,https://www.chet-notnkt8-grup.duckdns.org/,www.chet-notnkt8-grup.duckdns.org,33
4,http://veromcertiom.ddns.net/connexion/certico...,veromcertiom.ddns.net,21
...,...,...,...
10624,http://sites.google.com/site/habbotuttogratis,sites.google.com,16
10625,http://www.habbocreditosparati.blogspot.com/,www.habbocreditosparati.blogspot.com,36
10626,http://creditiperhabbogratissicuro100.blogspot...,creditiperhabbogratissicuro100.blogspot.com,43
10627,http://mundovirtualhabbo.blogspot.com/2009_01_...,mundovirtualhabbo.blogspot.com,30


In [8]:
for row in mixed_df[0]:
    mixed_df['Domain Length'] = mixed_df[0].str.len()
mixed_df

,0,Domain Length
0,goodrx.com,10
1,weather.com.cn,14
2,draftkings.com,14
3,suntimes.com,12
4,moe.gov.cn,10
...,...,...
178,services.runescape.com-ul.cc,28
179,nsu-amazon.1inc.top,19
180,80fxv9.cn,9
181,nsu-amazon.3inc.top,19


In [38]:
for row in tranco_df[1]:
    tranco_df['Domain Length'] = tranco_df[1].str.len()
tranco_df
# mine was taking ages to finish this. I never acually got it to finish running. is it fast on yours?

KeyboardInterrupt: 

In [46]:
# These for loops do the digit %, unique chars, hyphen count, and dot count in the domains 
digit_count_list = []
dot_count_list = []
hyphen_list = []
for x in benign_df[0]:
    z = 0
    q = 0
    h = 0
    for y in x:
        if y == '0' or y == "1" or y == "2"or y == "3" or y == "4" or y == "5" or y == "6" or y == "7" or y == "8" or y == "9":
            z = z + 1
        elif y == ".":
            q = q + 1
        elif y == "-":
            h = h + 1
    hyphen_list.append(h)
    dot_count_list.append(q)
    percent = (z / len(x)) * 100
    digit_count_list.append(percent)
benign_df["unique chars"] = benign_df[0].apply(lambda x: len(np.unique([*x])))
benign_df["hyphen count"] = hyphen_list
benign_df["Dot count"] = dot_count_list
benign_df["Digit %"] = digit_count_list
benign_df

,0,unique chars,Digit %,Dot count,hyphen count
0,google.com,7,0.0,1,0
1,facebook.com,9,0.0,1,0
2,a-msedge.net,10,0.0,1,1
3,youtube.com,9,0.0,1,0
4,microsoft.com,9,0.0,1,0
...,...,...,...,...,...
4958,bethesda.net,9,0.0,1,0
4959,shopifysvc.com,11,0.0,1,0
4960,instana.io,7,0.0,1,0
4961,oanda.com,7,0.0,1,0


In [47]:
# These for loops do the digit %, unique chars, hyphen count, and dot count in the domains 
malicious_df['Domain'] = malicious_df['url'].apply(get_fqdn)
dot_count_list = []
digit_count_list = []
hyphen_list = []
for x in malicious_df["Domain"]:
    z = 0
    q = 0
    h = 0
    for y in x:
        if y == '0' or y == "1" or y == "2"or y == "3" or y == "4" or y == "5" or y == "6" or y == "7" or y == "8" or y == "9":
            z = z + 1
        elif y == ".":
            q = q + 1
        elif y == "-":
            h = h + 1
    hyphen_list.append(h)
    dot_count_list.append(q)
    percent = (z / len(x)) * 100
    digit_count_list.append(percent)
malicious_df["unique chars"] = malicious_df["Domain"].apply(lambda x: len(np.unique([*x])))
malicious_df["hyphen count"] = hyphen_list
malicious_df["dot count"] = dot_count_list
malicious_df["Digit %"] = digit_count_list
malicious_df

,url,Domain,unique chars,Digit %,dot count,hyphen count
0,https://ervashipping.com/,ervashipping.com,14,0.000000,1,0
1,http://www.gabung-grupmabarcimbrut.duckdns.org/,www.gabung-grupmabarcimbrut.duckdns.org,18,0.000000,3,1
2,http://republik-jancuk.000webhostapp.com/,republik-jancuk.000webhostapp.com,21,9.090909,2,1
3,https://www.chet-notnkt8-grup.duckdns.org/,www.chet-notnkt8-grup.duckdns.org,17,3.030303,3,2
4,http://veromcertiom.ddns.net/connexion/certico...,veromcertiom.ddns.net,12,0.000000,2,0
...,...,...,...,...,...,...
10624,http://sites.google.com/site/habbotuttogratis,sites.google.com,10,0.000000,2,0
10625,http://www.habbocreditosparati.blogspot.com/,www.habbocreditosparati.blogspot.com,17,0.000000,3,0
10626,http://creditiperhabbogratissicuro100.blogspot...,creditiperhabbogratissicuro100.blogspot.com,19,6.976744,2,0
10627,http://mundovirtualhabbo.blogspot.com/2009_01_...,mundovirtualhabbo.blogspot.com,18,0.000000,2,0


In [48]:
# These for loops do the digit %, unique chars, hyphen count, and dot count in the domains 
dot_count_list = []
digit_count_list = []
hyphen_list = []
for x in mixed_df[0]:
    z = 0
    q = 0
    h = 0
    for y in x:
        if y == '0' or y == "1" or y == "2"or y == "3" or y == "4" or y == "5" or y == "6" or y == "7" or y == "8" or y == "9":
            z = z + 1
        elif y == ".":
            q = q + 1
        elif y == "-":
            h = h + 1
    hyphen_list.append(h)
    dot_count_list.append(q)
    percent = (z / len(x)) * 100
    digit_count_list.append(percent)
mixed_df["unique chars"] = mixed_df[0].apply(lambda x: len(np.unique([*x])))
mixed_df["hyphen count"] = hyphen_list
mixed_df["dot count"] = dot_count_list
mixed_df["Digit %"] = digit_count_list
mixed_df

,0,unique chars,Digit %,dot count,hyphen count
0,goodrx.com,8,0.000000,1,0
1,weather.com.cn,11,0.000000,2,0
2,draftkings.com,14,0.000000,1,0
3,suntimes.com,10,0.000000,1,0
4,moe.gov.cn,8,0.000000,2,0
...,...,...,...,...,...
178,services.runescape.com-ul.cc,15,0.000000,3,1
179,nsu-amazon.1inc.top,14,5.263158,2,1
180,80fxv9.cn,9,33.333333,1,0
181,nsu-amazon.3inc.top,14,5.263158,2,1


In [49]:
# These for loops do the digit %, unique chars, hyphen count, and dot count in the domains 
dot_count_list = []
digit_count_list = []
hyphen_list = []
for x in tranco_df[1]:
    z = 0
    q = 0
    h = 0
    for y in x:
        if y == '0' or y == "1" or y == "2"or y == "3" or y == "4" or y == "5" or y == "6" or y == "7" or y == "8" or y == "9":
            z = z + 1
        elif y == ".":
            q = q + 1
        elif y == "-":
            h = h + 1
    hyphen_list.append(h)
    dot_count_list.append(q)
    percent = (z / len(x)) * 100
    digit_count_list.append(percent)
tranco_df["unique chars"] = tranco_df[1].apply(lambda x: len(np.unique([*x])))
tranco_df["hyphen count"] = hyphen_list
tranco_df["dot count"] = dot_count_list
tranco_df["Digit %"] = digit_count_list
tranco_df

,0,1,unique chars,Digit %,Domain Length,dot count,hyphen count
0,1,google.com,7,0.0,10,1,0
1,2,facebook.com,9,0.0,12,1,0
2,3,a-msedge.net,10,0.0,12,1,1
3,4,youtube.com,9,0.0,11,1,0
4,5,microsoft.com,9,0.0,13,1,0
...,...,...,...,...,...,...,...
3648682,3648683,jeventanddesign.com,14,0.0,19,1,0
3648683,3648684,medi-verbund.de,11,0.0,15,1,1
3648684,3648685,stjudemedical.net,13,0.0,17,1,0
3648685,3648686,dsa-vorschau.de,12,0.0,15,1,1
